# FrozenLake 8x8 - QTable

**(This doesn't work yet)**

This environment has 256 states. Because of this E-Greedy Agent takes a lot of episoded to learn the correct policy.
To speed it up in this notebook we use UCB Agent. Based on Upper-Confidence-Bound Action Selection.
For more information check [Sutton book](http://incompleteideas.net/sutton/book/bookdraft2016sep.pdf).

For this tutorial we will use [Frozen Lake 8x8](https://gym.openai.com/envs/FrozenLake8x8-v0).

### Solved
FrozenLake is solved if moving average over window size 100 is >= 0.99

In [1]:
import logging
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym import wrappers

logging.getLogger('gym').setLevel(logging.WARNING)

## Helper functions

In [2]:
def moving_average(xs, n=100):
    ret = np.cumsum(xs, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def find_index(xs, v):
    """Find index of the first value equal or greater then v"""
    for i in range(len(xs)):
        if xs[i] >= v:
            return i
    return -1

def run_env(agent, env, num_episodes):
    rewards = []
    for episode in range(num_episodes):
        s = env.reset()
        total_reward = 0
        while True:
            a = agent.choose_action(s, episode)
            s2, reward, done, _ = env.step(a)        
            agent.learn(s, a, reward, s2)
            total_reward += reward
            s = s2
            if done:
                agent.end_state(s2, reward)               
                break
        rewards.append(total_reward)

    return rewards

## Explore environment

In [19]:
env = gym.make('FrozenLake8x8-v0')        
states = np.zeros(64)
for i in range(1000):
    s = env.reset()
    action = 3
    s2, reward, done, _ = env.step(action)
    states[s2] += 1
#     print('%d: state=%d, action=%d, end_state=%d' % (i, s, action, s2))
print(states.reshape((8,8)))
env.close()

[[ 676.  324.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.    0.    0.    0.]]


## UCB Table Agent

The UCB tries to estimate the Q function value and variance. 
Then action is selected based on both value and variance.

The formula to select action:

$$ A_t = argmax_{a} \left[ Q_t(a) + c \sqrt{ \frac{log(t)}{N_t(a)} } \right] $$

Where:
  * $\sqrt{ \frac{log(t)}{N_t(a)} }$ - Is variance (uncertainty) of action value
  * $N_t(a)$ - How many times this action was selected
  * c - controls the degree of exploration
  
The formula to learn:

$$ Q'_{a} = \frac{Q_{a}*(n-1) + r}{n} $$

In [3]:
class UCBAgent:
    
    def __init__(self, num_states, num_actions, alpha, c=2.0):
        self.qtable = np.zeros([num_states, num_actions])
        self.num_actions = num_actions
        self.c = c
        self.gamma = 0.99
        self.alpha = alpha
        self.counters = np.zeros([num_states, num_actions])
        
    def choose_action(self, state, _):
        if np.min(self.counters[state,:]) == 0:
            return np.argmin(self.counters[state,:])
        else:
            t = np.sum(self.counters[state,:])
            actions = self.c * np.sqrt(np.log(t)/self.counters[state,:])
            return np.argmax(self.qtable[state,:] + actions)
        
    def learn(self, state, action, reward, next_state):
        """ Update state using SARSA(0)
            Also add -1 to the reward to let agent learn shortest path
        """
        next_action = self.choose_action(next_state, 0)
        q = reward + self.gamma * self.qtable[next_state,next_action]
        self.qtable[state, action] += self.alpha * (q-self.qtable[state, action])
        self.counters[state, action] += 1    
        
    def end_state(self, state, reward):
        """ Last state reached. 
        """
        if reward > 0:
            self.qtable[state, :] = 1
        else:
            self.qtable[state, :] = -1
        
    def values(self):
        return self.qtable.max(axis=1)        
    
    def policy(self):
        return self.qtable.argmax(axis=1)            

## Train agent

Messing with the rewards:

  * If we don't change reward then the probability of randomly finding solution is very low. 
    And the QTable learning will be very slow. And since there are 256 states, the propagation of
    value is very slow
  * By modifying reward we can try to at least try to not fall into the hole

In [9]:
# Learning parameters
learning_rate = 0.1
solved_score = 0.99
# Environment
env = gym.make('FrozenLake8x8-v0')
# Agent
agent = UCBAgent(env.observation_space.n, env.action_space.n, learning_rate)
found = 0
for episode in range(int(1e6)):
    s = env.reset()
    while True:
        a = agent.choose_action(s, episode)
        s2, reward, done, _ = env.step(a)   
        agent.learn(s, a, reward, s2)
        s = s2
        if done:
            agent.end_state(s, reward)
            break
    if reward > 0:
        found += 1
        if found > 1:
            break

print('Found solution %d times' % found)        
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)
print(agent.values().reshape(8,8))
print("Policy:")
print(agent.policy().reshape(8,8))
env.close()    

Found solution 2 times
[[-0.579 -0.596 -0.582 -0.554 -0.492 -0.344 -0.223 -0.095]
 [-0.603 -0.613 -0.627 -0.622 -0.558 -0.434 -0.229 -0.138]
 [-0.623 -0.663 -0.69  -1.    -0.597 -0.473 -0.285 -0.136]
 [-0.682 -0.697 -0.741 -0.762 -0.442 -1.    -0.217 -0.087]
 [-0.679 -0.784 -0.861 -1.    -0.059 -0.039 -0.114 -0.069]
 [-0.555 -1.    -1.    -0.099 -0.038 -0.004 -1.    -0.001]
 [-0.184 -1.     0.     0.    -1.    -0.043 -1.     0.199]
 [-0.046 -0.013  0.    -1.     0.     0.     0.     1.   ]]
Policy:
[[3 3 3 3 3 3 2 2]
 [3 0 3 3 2 1 2 2]
 [3 0 0 0 2 3 2 2]
 [3 0 3 1 0 0 2 2]
 [0 3 2 0 2 1 3 2]
 [0 0 0 0 2 0 0 2]
 [0 0 0 0 0 3 0 2]
 [1 1 0 0 0 0 0 0]]


In [ ]:
# Learning parameters
learning_rate = 0.01
num_episodes = int(1e7)
solved_score = 0.99
# Environment
env = gym.make('FrozenLake8x8-v0')
# Agent
agent = UCBAgent(env.observation_space.n, env.action_space.n, learning_rate)
# run simulation
rewards = run_env(agent, env, num_episodes)
env.close()
# Show summary
averaged_rewards = moving_average(rewards)    
idx = find_index(averaged_rewards, solved_score)
print('Max score: %f' % np.max(averaged_rewards))
if idx >= 0:
    print('Solved after {} episodes'.format(idx+1))
else:
    print('Not solved')

plt.plot(averaged_rewards)
plt.show()
print("Max Q value:")
print(agent.values().reshape(8,8))
print("Policy:")
print(agent.policy().reshape(8,8))